In [ ]:
# 事前にGCPの認証が必要で、認証方法は環境によって異なる
# colabの場合:
#   セルで下記を実行
#     from google.colab import auth
#     auth.authenticate_user()
#     %env GCLOUD_PROJECT=GCPのプロジェクトID
# PC等のローカル環境の場合:
#   初回のみ、https://cloud.google.com/sdk/docs/install-sdk からgcloud CLIをインストールし、gcloud initを実行
%load_ext google.cloud.bigquery

# 12章 日時
## 12-1 日時型への変換
### Q: 文字列の日時を日時型に変換
#### Awesome

In [ ]:
%%bigquery
with
values as (
    select "2023-04-01 10:11:30" as datetime_str
)

select
    -- （1） 文字列の左側10文字を取り出してdate型へ変換
    parse_date("%Y-%m-%d", left(datetime_str, 10)) as date,
    -- （2） 文字列の右側8文字を取り出してtime型へ変換
    parse_time("%H:%M:%S", right(datetime_str, 8)) as time,
    -- （3） datetime型へ変換
    parse_datetime("%Y-%m-%d %H:%M:%S", datetime_str) as datetime,
    -- （4） timestamp型へ変換
    parse_timestamp("%Y-%m-%d %H:%M:%S", datetime_str) as timestamp,
    -- （5） timestamp型をJSTのdatetime型へ変換
    datetime(parse_timestamp("%Y-%m-%d %H:%M:%S", datetime_str), "Asia/Tokyo")
        as datetime_jst,
    -- （6） timestamp型をPSTのdatetime型へ変換
    datetime(
        parse_timestamp("%Y-%m-%d %H:%M:%S", datetime_str),
        "America/Los_Angeles"
    ) as datetime_pst
from values

## 12-2 日時型の列を用いた行の抽出
### Q: 指定期間に予約された予約履歴の抽出


#### Not Awesome

In [ ]:
%%bigquery
select *
from example.reservation
where reserved_at between "2019-07-01" and "2019-08-31"

#### Awesome

In [ ]:
%%bigquery
select *
from example.reservation
where date(reserved_at, "Asia/Tokyo") between "2019-07-01" and "2019-08-31"

## 12-3 日時要素の抽出
### Q: 予約日時の日時要素を抽出
#### Awesome

In [ ]:
%%bigquery
select
    reserved_at,
    -- （1） 年を抽出
    extract(year from reserved_at at time zone "Asia/Tokyo") as reserved_at_year,
    -- （2） 月を抽出
    extract(month from reserved_at at time zone "Asia/Tokyo") as reserved_at_month,
    -- （3） 日を抽出
    extract(day from reserved_at at time zone "Asia/Tokyo") as reserved_at_day,
    -- （4） 曜日を抽出（週の最初を日曜日とした1〜7の数字）
    extract(dayofweek from reserved_at at time zone "Asia/Tokyo")
        as reserved_at_dayofweek,
    -- （5） 週を抽出（ISO 8601形式の週番号）
    extract(isoweek from reserved_at at time zone "Asia/Tokyo")
        as reserved_at_isoweek,
    -- （6） 時を抽出
    extract(hour from reserved_at at time zone "Asia/Tokyo") as reserved_at_hour,
    -- （7） 分を抽出
    extract(minute from reserved_at at time zone "Asia/Tokyo") as reserved_at_minute,
    -- （8） 秒を抽出
    extract(second from reserved_at at time zone "Asia/Tokyo") as reserved_at_second,
    -- （9） 年月を抽出
    format_timestamp("%Y-%m", reserved_at, "Asia/Tokyo") as reserved_at_ym
from example.reservation

## 12-4 日時の丸め処理
### Q: 予約日時の切り捨て
#### Awesome

In [ ]:
%%bigquery
select
    reserved_at,
    -- （1） 年に切り捨て
    timestamp_trunc(reserved_at, year, "Asia/Tokyo") as reserved_at_year,
    -- （2） 四半期に切り捨て
    timestamp_trunc(reserved_at, quarter, "Asia/Tokyo") as reserved_at_quarter,
    -- （3） 月に切り捨て
    timestamp_trunc(reserved_at, month, "Asia/Tokyo") as reserved_at_month,
    -- （4） 日に切り捨て
    timestamp_trunc(reserved_at, day, "Asia/Tokyo") as reserved_at_day,
    -- （5） 時に切り捨て
    timestamp_trunc(reserved_at, hour, "Asia/Tokyo") as reserved_at_hour,
    -- （6） 分に切り捨て
    timestamp_trunc(reserved_at, minute, "Asia/Tokyo") as reserved_at_minute
from example.reservation

## 12-5 日時の加減算
### Q: 予約日時に一定時間を加算
#### Awesome

In [ ]:
%%bigquery
select
    reserved_at,
    -- （1） 30分を加算
    reserved_at + interval 30 minute as reserved_at_add30min,
    -- （2） 1時間を加算
    reserved_at + interval 1 hour as reserved_at_add1h,
    -- （3） 1日を加算
    reserved_at + interval 1 day as reserved_at_add1d,
    -- （4） 1ヶ月を加算
    date_add(date(reserved_at, "Asia/Tokyo"), interval 1 month) as reserved_at_add1m,
    -- （5） 1年を加算
    date_add(date(reserved_at, "Asia/Tokyo"), interval 1 year) as reserved_at_add1y
from example.reservation

### Q: 予約日時とチェックイン日の時間差を計算
#### Awesome

In [ ]:
%%bigquery
select
    reserved_at,
    checkin_date,
    -- （1） 年数差を計算
    cast(floor(
        (
            date_diff(
                date(checkin_date, "Asia/Tokyo"),
                date(reserved_at, "Asia/Tokyo"),
                month
            )
            - case when
                extract(day from checkin_date at time zone "Asia/Tokyo")
                < extract(day from reserved_at at time zone "Asia/Tokyo")
                then 1
            else 0 end
        ) / 12
    ) as int) as diff_years,
    -- （2） 月数差を計算
    date_diff(
        date(checkin_date, "Asia/Tokyo"),
        date(reserved_at, "Asia/Tokyo"),
        month
    )
    - case when
        extract(day from checkin_date at time zone "Asia/Tokyo")
        < extract(day from reserved_at at time zone "Asia/Tokyo")
        then 1
    else 0 end as diff_months,
    -- （3） 日数差を計算
    timestamp_diff(checkin_date, reserved_at, day) as diff_days,
    -- （4） 時間差を計算
    timestamp_diff(checkin_date, reserved_at, hour) as diff_hours,
    -- （5） 分差を計算
    timestamp_diff(checkin_date, reserved_at, minute) as diff_minutes,
    -- （6） 秒差を計算
    timestamp_diff(checkin_date, reserved_at, second) as diff_seconds
from example.reservation

## 12-6 日時の数値化
### Q: チェックイン日の休日フラグの付与
#### Awesome

In [ ]:
%%bigquery
with
-- （1） 日付マスタの作成
date_master as (
    select
        dt,
        is_day_off,
        -- （1）-4 次の日の休日フラグを取得して休日前日フラグを作成
        lead(is_day_off) over (order by dt) as is_day_before_day_off
    from (
        select
            dt,
            -- （1）-3 休日フラグの作成
            case
                -- 祝日の場合は1
                when holiday_name is not null then 1
                -- 曜日が土日の場合は1
                when extract(dayofweek from dt) in (1, 7) then 1
                -- 上記以外は0
                else 0
            end as is_day_off
        -- （1）-1 連続した日付の生成
        from unnest(generate_date_array("2014-01-01", "2019-12-31")) as dt
        -- （1）-2 祝日マスタの結合
        left join example.holiday on dt = date(holiday_date)
    )
)

select * except (dt)
from example.reservation
-- （2） checkin_dateを用いて日付マスタをjoin
left join date_master on date(checkin_date, "Asia/Tokyo") = dt